In [1]:
import pandas as pd
import datetime as datetime
import numpy as np
import copy
from RSU_functions import * 

In [2]:
# If you want to change your marginal income tax (above 72k of taxable income, it is at 41%). Not really needed
tax_info_dict = {'TMI_IR': 0.3, 'cotisation': 0.172, 'flat_tax_plus_value': 0.3}

# Put here all sales and vesting event from EquatePlus
please put all event until end of 2020. If you have sales in 2019, either rely on the built in function or please du check yourself how you reported them before. If you sold everything you had in 2019, it will work, otherwise it will work too but I might link a sale with a vesting date you already declared as sold.

In [3]:

vesting_event = [{'date':datetime.datetime(2019,12,13),'amount':225,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,3,13),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,6,15),'amount':29,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,9,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                {'date':datetime.datetime(2020,12,14),'amount':28,'macron_law_id':0, 'event':'vesting'},
                ]
selling_event = [
    {'date':datetime.datetime(2020,8,3),'amount':282, 'event':'sale'},
    {'date':datetime.datetime(2020,12,16),'amount':56, 'event':'sale'}
]

Computation occurs here, please just run these cells

In [4]:
all_events_sorted = sorted(vesting_event + selling_event, key = lambda k: k['date'])

In [5]:
portfolio = {}
portfolio['available_stock'] = []
portfolio['sale_event_recap'] = []
for event in all_events_sorted:
    if event['event'] == 'vesting':
        portfolio['available_stock'].append(copy.deepcopy(event))
    if event['event'] == 'sale':
        available_stock_after_transaction, sale_event_recap = get_sales_result(event, portfolio,
                                                                               matching_method = get_sale_order_from_optionality)
        portfolio['available_stock'] = available_stock_after_transaction
        portfolio['sale_event_recap'].append(sale_event_recap)
        

# Results to put in your taxation returns
Ici, toute les infos sont printer pour remplir le formulaire No 2074. 
pour 511 mettre Action criteo et equateplus.

#En cas de moins value, toujours pas clair s il faut remplir avec un resultat negatif,
ou sil faut remplir en changeant le 'prix ou valeur  d aquisition unitaire' et le mettre egal au prix de cession

Rebate ne semblent pas applicables non plus. Please help

In [6]:

tax = 0
Capital_gain = 0
vesting_gain_after_rebate = 0
vesting_gain_before_rebate = 0
total_selling_price = 0
for event in portfolio['sale_event_recap']:
    for ev in event:
        if ev['date de la cession (513)'].year ==2020:
            print('\n ----------------NOUVELLE TRANSACTION ---------- \n')
            for key, value in ev.items():
                print(  key, value)
            tax +=ev['tax']
            Capital_gain += ev['plus_value_amount']
            vesting_gain_after_rebate += ev['vesting_amount_with_moins_and_rebate']
            vesting_gain_before_rebate += ev['vesting_amount_with_moins_value']
            total_selling_price += ev['montant global (516 et 518)']
        



 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2020-08-03 00:00:00
valeur unitaire de la cession (514) 11.572573767695719
nombre de titres cedes (515) 28
montant global (516 et 518) 324.03206549548014
prix ou valeur acquisition unitaire (520) 7.47478386167147
prix daquisition global (521 et 523) 209.29394812680115
resultat 114.73811736867897
vesting_amount_with_moins_value 209.29394812680115
vesting_amount_with_moins_and_rebate 209.29394812680115
plus_value_amount 114.73811736867897
rebate 1
tax 133.20817872645384

 ----------------NOUVELLE TRANSACTION ---------- 

date de la cession (513) 2020-08-03 00:00:00
valeur unitaire de la cession (514) 11.572573767695719
nombre de titres cedes (515) 29
montant global (516 et 518) 335.60463926317584
prix ou valeur acquisition unitaire (520) 10.868212920998845
prix daquisition global (521 et 523) 315.1781747089665
resultat 20.42646455420934
vesting_amount_with_moins_value 315.1781747089665
vesting_amount_with_moins_

In [7]:
if vesting_gain_after_rebate > 300000:
    print('fill 1TT, form 2042 C with ', vesting_gain_after_rebate - 300000 )
print('fill 1TZ, form 2042 C with ', vesting_gain_after_rebate)
print('fill 1UZ , form 2042 C with the rebate in euro (might not be exactly how its done...)', vesting_gain_before_rebate-vesting_gain_after_rebate )
print('fill 3VG , form 2042 C with capital gain', Capital_gain )

print('You sold for {} this amount of stocks'.format(np.round(total_selling_price,2)))
print('You will pay around  {} this amount of tax, sorry'.format(np.round(tax ,2)))

fill 1TZ, form 2042 C with  3857.266008651111
fill 1UZ , form 2042 C with the rebate in euro (might not be exactly how its done...) 0.0
fill 3VG , form 2042 C with capital gain 267.632232923502
You sold for 4124.9 this amount of stocks
You will pay around  1900.92 this amount of tax, sorry
